In [2]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, 6d88f759-b514-4376-8007-77026c5b9e42, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, 44079834-0692-41ae-8bba-05efa1214c8d)

In [3]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 6d88f759-b514-4376-8007-77026c5b9e42, 5, Finished, Available)

In [4]:
# Import the model and configure the input and output columns
model = (AnalyzeText()
         .setTextCol("description")
         .setKind("SentimentAnalysis")
         .setOutputCol("response")
         .setErrorCol("error"))

StatementMeta(, 6d88f759-b514-4376-8007-77026c5b9e42, 6, Finished, Available)

In [5]:
# Apply the model to our dataframe
result = model.transform(df)

StatementMeta(, 6d88f759-b514-4376-8007-77026c5b9e42, 7, Finished, Available)

In [6]:
display(result)

StatementMeta(, 6d88f759-b514-4376-8007-77026c5b9e42, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9581d0ed-eebf-4015-8cde-0a754f162600)

In [7]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, 6d88f759-b514-4376-8007-77026c5b9e42, 9, Finished, Available)

In [8]:
display(sentiment_df)

StatementMeta(, 6d88f759-b514-4376-8007-77026c5b9e42, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, 26786962-13e6-45f1-b328-01cc1932cfef)

In [9]:
sentiment_df_final = sentiment_df.drop("error", "response")

StatementMeta(, 6d88f759-b514-4376-8007-77026c5b9e42, 11, Finished, Available)

In [10]:
display(sentiment_df_final)

StatementMeta(, 6d88f759-b514-4376-8007-77026c5b9e42, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9b26383b-3081-4549-bd1c-023760e5a849)

In [17]:
from pyspark.sql.functions import col,to_date

sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "MM-dd-yyyy"))

StatementMeta(, 6d88f759-b514-4376-8007-77026c5b9e42, 19, Finished, Available)

### Type 1 Merge

In [11]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_analysis'
    sentiment_df_final.write.format("delta").saveAsTable(table_name)
except AnalysisException:
    print("Table Already Exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""
        MERGE INTO {table_name} target_table
        USING vw_sentiment_df_final source_view
        ON source_view.url = target_table.url
        WHEN MATCHED AND
            source_view.title <> target_table.title OR
            source_view.description <> target_table.description OR
            source_view.category <> target_table.category OR
            source_view.image <> target_table.image OR
            source_view.provider <> target_table.provider OR
            source_view.datePublished <> target_table.datePublished
        THEN UPDATE SET *

        WHEN NOT MATCHED THEN INSERT *

    """)


StatementMeta(, 6d88f759-b514-4376-8007-77026c5b9e42, 13, Finished, Available)

Table Already Exists
